In [43]:
import pandas as pd
pd.options.mode.copy_on_write = True
import numpy as np
import ast
import  pickle
from uszipcode import SearchEngine


In [2]:
df = pd.read_csv('data/data.csv',
                 usecols=['state','schools'])
df.head(3)

,schools,state
0,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",NC
1,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...",WA
2,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...",CA


In [3]:
df['schools_ast'] = df['schools'].apply(lambda x: ast.literal_eval(x)[0])

In [4]:
url = 'https://www.bu.edu/brand/guidelines/editorial-style/us-state-abbreviations/'
tables = pd.read_html(url)

short_state_name = set(tables[0][2][1:])
print(short_state_name)

{'AR', 'UT', 'NM', 'IN', 'RI', 'NV', 'TX', 'IA', 'AZ', 'MO', 'VA', 'IL', 'CA', 'HI', 'MN', 'LA', 'TN', 'KY', 'NE', 'OH', 'WA', 'FL', 'CT', 'ME', 'SD', 'WY', 'OK', 'SC', 'NJ', 'CO', 'VT', 'MS', 'ID', 'NH', 'KS', 'MA', 'GA', 'MT', 'DE', 'OR', 'PA', 'NC', 'NY', 'AL', 'AK', 'ND', 'WI', 'MD', 'WV', 'DC', 'MI'}


In [5]:
df['state'] = df['state'].str.upper()
df['state_st'] = df['state'].apply(lambda x: 0 if x in short_state_name else 1)
print(df['state_st'].sum())
df[df['state_st']==1]
df = df[df['state_st']!=1]
df = df.drop('state_st',axis=1)

3


In [6]:
df['school_rating'] = df['schools_ast'].apply(lambda x: x['rating'])

In [7]:
df['school_name']  = df['schools_ast'].apply(lambda x: x['name'])
df['school_name']

0         [Southern Pines Elementary School, Southern Mi...
1         [East Valley High School&Extension, Eastvalley...
2         [Paul Revere Middle School, Brentwood Science ...
3         [Mcculloch Intermediate School, Bradfield Elem...
4         [Southwest Middle School, Bayside High School,...
                                ...                        
377180    [Air Base Elementary School, Dr Michael M. Kro...
377181    [Hope College Prep High School, Lake View High...
377182    [Ps 48 William Wordsworth, Jhs 8 Richard S Gro...
377183    [Hiliard El, Forest Brook Middle, North Forest...
377184    [Mary Lou Hartman, Woodlake Hills Middle Schoo...
Name: school_name, Length: 377182, dtype: object

In [8]:
rating_invalid_search = []
df['school_rating'].apply(lambda x: rating_invalid_search.extend(x))
print(set(rating_invalid_search))

{'', '9/10', '7/10', '4', '10/10', '8', '2', 'None/10', '3', '2/10', '4/10', '1', '7', '9', '8/10', '1/10', '10', '5', '6', 'NR', 'NA', '6/10', '3/10', '5/10'}


In [9]:
df['temp'] = df.apply(lambda row: dict(zip(row['school_name'],row['school_rating'])),axis=1)
df['temp']

0         {'Southern Pines Elementary School': '4', 'Sou...
1         {'East Valley High School&Extension': '4/10', ...
2         {'Paul Revere Middle School': '8/10', 'Brentwo...
3         {'Mcculloch Intermediate School': '9/10', 'Bra...
4         {'Southwest Middle School': '4/10', 'Bayside H...
                                ...                        
377180    {'Air Base Elementary School': '10/10', 'Dr Mi...
377181    {'Hope College Prep High School': '1/10', 'Lak...
377182    {'Ps 48 William Wordsworth': '5/10', 'Jhs 8 Ri...
377183    {'Hiliard El': 'NA', 'Forest Brook Middle': 'N...
377184    {'Mary Lou Hartman': '5/10', 'Woodlake Hills M...
Name: temp, Length: 377182, dtype: object

In [12]:
def school_temp(x):
    for i,j in x.items():
        temp_school_name[i].append(j)
    return x

In [13]:
# short_state_name

df_state_school = pd.DataFrame(columns = ['state','school_name','rating'])

for state in short_state_name:
    df_state_school = pd.DataFrame()
    df_temp = df[df.state==state][['temp','school_name','school_rating']].copy()

    school_name = []
    df_temp['school_name'].apply(lambda x: school_name.extend(x))
    school_name = set(school_name)

    temp_school_name = {}
    for name in school_name:
        temp_school_name[name] = []

    df_temp['temp'].apply(school_temp)

    for k, v in temp_school_name.items():
        ss = pd.DataFrame({'state':state,'school_name':k,'rating':v},columns = ['state','school_name','rating'])
        ss.rating = ss.rating.replace({'NR':'None','NA':'None'})
        ss.rating = ss.rating.apply(lambda x:str(x).split(sep='/')[0] if '/' in str(x) else x)
        ss.rating = ss.rating.apply(lambda x: np.NaN if x is None or x is np.NaN or x == 'None' else int(x))

        df_state_school = pd.concat([df_state_school,ss ],ignore_index=True)


df_state_school_group = df_state_school.groupby(['state','school_name'])['rating'].agg('describe')